# MeerKAT conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.17 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_meerkat.ms")

small_meerkat.ms.zip:   0%|          | 0.00/2.28M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

ms_file = "small_meerkat.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_meerkat.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_name=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,"small_meerkat_ddi_0_intent_CALIBRATE_BANDPASS,...",0,"CALIBRATE_BANDPASS,CALIBRATE_FLUX",0,J1939-6342,3.265869e+09,3.276337e+09
1,small_meerkat_ddi_0_intent_TARGET_field_id_2,0,TARGET,2,J0358-8103,3.265869e+09,3.276337e+09
2,"small_meerkat_ddi_0_intent_CALIBRATE_PHASE,CAL...",0,"CALIBRATE_PHASE,CALIBRATE_AMPLI",1,J1619-8418,3.265869e+09,3.276337e+09


In [5]:
ps.get(0)

<xarray.Dataset>
Dimensions:                     (time: 74, baseline_id: 6, frequency: 50,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 dask.array<chunksize=(6,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 dask.array<chunksize=(6,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 5
  * frequency                   (frequency) float64 3.266e+09 ... 3.276e+09
  * polarization                (polarization) <U2 'XX' 'XY' 'YX' 'YY'
  * time                        (time) float64 1.7e+09 1.7e+09 ... 1.7e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(20, 6), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(20, 6, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(20, 6), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(20, 6, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(20, 6, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(20, 6, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          0
    field_info:   {'code': 'T', 'delay_direction': {'attrs': {'frame': 'FK5',...
    intent:       CALIBRATE_BANDPASS,CALIBRATE_FLUX
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 3, xyz_la...

In [6]:
ps.get(0).attrs['antenna_xds']

<xarray.Dataset>
Dimensions:        (antenna_id: 3, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 0 1 2
    mount          (antenna_id) <U6 dask.array<chunksize=(3,), meta=np.ndarray>
    name           (antenna_id) <U4 dask.array<chunksize=(3,), meta=np.ndarray>
    station        (antenna_id) <U4 dask.array<chunksize=(3,), meta=np.ndarray>
    type           (antenna_id) <U12 dask.array<chunksize=(3,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 dask.array<chunksize=(3,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 dask.array<chunksize=(3, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 dask.array<chunksize=(3, 3), meta=np.ndarray>